# _CONTROL COMERCIAL PARA ASESORES_

### _CUENTA CONVERGENTE_

In [1]:
from datetime import datetime

# =====================================
# FUNCIÓN CARGA DE DATOS
# =====================================

def cargar_asesor():
    asesor = {}

    print("=====================================")
    print("CONTROL COMERCIAL PARA ASESORES")
    print("=====================================\n")

    asesor["usuario"] = input("Ingrese nombre del asesor: ")
    asesor["fecha"] = datetime.now().strftime("%d/%m/%Y %H:%M")

    print("\n📅 Registro")
    print("Asesor:", asesor["usuario"])
    print("Fecha:", asesor["fecha"])

    print("\n--- OBJETIVOS MENSUALES (configurables) ---")
    asesor["objetivos"] = {
        "portabilidad": int(input("Objetivo Portabilidad: ")),
        "linea_nueva": int(input("Objetivo Línea Nueva: ")),
        "baf": int(input("Objetivo BAF: "))
    }

    print("\n--- PORTABILIDAD POR PLAN ---")
    asesor["portabilidad_planes"] = {
        "4gb": int(input("Portabilidades 4GB activadas: ")),
        "otros": int(input("Portabilidades otros planes activadas: "))
    }

    print("\n--- VENTAS ACTIVADAS ---")
    asesor["activadas"] = {
        "linea_nueva": int(input("Líneas nuevas activadas: ")),
        "baf": int(input("BAF activados: "))
    }

    print("\n--- DÍAS HÁBILES ---")
    asesor["dias"] = {
        "general": int(input("Días hábiles (Portabilidad + Línea Nueva): ")),
        "baf": int(input("Días hábiles BAF: "))
    }

    return asesor


# =====================================
# SEMÁFORO GENERAL
# =====================================

def calcular_semaforo(ventas_por_dia):
    if ventas_por_dia <= 0.5:
        return "🟢 VERDE"
    elif ventas_por_dia <= 1:
        return "🟡 AMARILLO"
    else:
        return "🔴 ROJO"


# =====================================
# SEMÁFORO LÍNEA NUEVA
# =====================================

def semaforo_linea_nueva(objetivo, activadas, dias):
    faltan = max(0, objetivo - activadas)
    por_dia = round(faltan / dias, 2) if dias > 0 else 0
    semaforo = calcular_semaforo(por_dia)

    print("\n📊 LÍNEA NUEVA")
    print(f"Objetivo: {objetivo}")
    print(f"Activadas: {activadas}")
    print(f"Faltan: {faltan}")
    print(f"Debe vender por día: {por_dia}")
    print(f"Semáforo: {semaforo}")


# =====================================
# SEMÁFORO PORTABILIDAD + MIX 4GB
# =====================================

def semaforo_portabilidad(asesor):
    objetivo = asesor["objetivos"]["portabilidad"]
    dias = asesor["dias"]["general"]

    porta_4gb = asesor["portabilidad_planes"]["4gb"]
    porta_otros = asesor["portabilidad_planes"]["otros"]

    total_porta = porta_4gb + porta_otros
    max_4gb = int(total_porta * 0.25)

    print("\n📊 PORTABILIDAD")
    print(f"Total ventas: {total_porta}")
    print(f"4GB: {porta_4gb}")
    print(f"Otros planes: {porta_otros}")
    print(f"Máx 4GB permitido (25%): {max_4gb}")

    if porta_4gb > max_4gb:
        exceso = porta_4gb - max_4gb
        print(f"❌ Exceso 4GB (+{exceso}) → No comisiona portabilidad")
        mix_ok = False
        porta_validas = max_4gb + porta_otros
    else:
        print("✅ Mix 4GB correcto")
        mix_ok = True
        porta_validas = total_porta

    faltan = max(0, objetivo - porta_validas)
    por_dia = round(faltan / dias, 2) if dias > 0 else 0
    semaforo = calcular_semaforo(por_dia)

    print(f"Ventas válidas: {porta_validas}")
    print(f"Faltan: {faltan}")
    print(f"Debe vender por día: {por_dia}")
    print(f"Semáforo: {semaforo}")

    return porta_validas, mix_ok


# =====================================
# SEMÁFORO BAF
# =====================================

def semaforo_baf(objetivo, activadas, dias):
    faltan = max(0, objetivo - activadas)
    por_dia = round(faltan / dias, 2) if dias > 0 else 0
    semaforo = calcular_semaforo(por_dia)

    print("\n📊 BAF")
    print(f"Objetivo: {objetivo}")
    print(f"Activados: {activadas}")
    print(f"Faltan: {faltan}")
    print(f"Debe vender por día: {por_dia}")
    print(f"Semáforo: {semaforo}")


# =====================================
# EVALUACIÓN DE COMISIONES
# =====================================

def evaluar_comisiones(asesor, porta_validas, mix_ok):
    print("\n===================================")
    print("EVALUACIÓN DE COMISIONES")
    print("===================================")

    if not mix_ok:
        print("❌ NO CALIFICA A COMISIONES (Exceso de 4GB)")
        return

    comisiones = {
        "COMISIÓN 60% + SÁBADOS LIBRES": {
            "portabilidad": 17,
            "linea_nueva": 2,
            "baf": 3
        },
        "COMISIÓN 100% + SÁBADOS LIBRES": {
            "portabilidad": 28,
            "linea_nueva": 2,
            "baf": 5
        }
    }

    for nombre, req in comisiones.items():
        print(f"\n🎯 {nombre}")
        cumple = True

        if porta_validas < req["portabilidad"]:
            print(f"❌ PORTABILIDAD: {porta_validas}/{req['portabilidad']}")
            cumple = False
        else:
            print(f"✅ PORTABILIDAD: {porta_validas}/{req['portabilidad']}")

        for tipo in ["linea_nueva", "baf"]:
            activadas = asesor["activadas"][tipo]
            if activadas < req[tipo]:
                print(f"❌ {tipo.upper()}: {activadas}/{req[tipo]}")
                cumple = False
            else:
                print(f"✅ {tipo.upper()}: {activadas}/{req[tipo]}")

        if cumple:
            print("🏆 CUMPLE COMISIÓN")
        else:
            print("⛔ NO CUMPLE COMISIÓN")


# =====================================
# EJECUCIÓN PRINCIPAL (JUPYTER)
# =====================================

asesor = cargar_asesor()

porta_validas, mix_ok = semaforo_portabilidad(asesor)

semaforo_linea_nueva(
    asesor["objetivos"]["linea_nueva"],
    asesor["activadas"]["linea_nueva"],
    asesor["dias"]["general"]
)

semaforo_baf(
    asesor["objetivos"]["baf"],
    asesor["activadas"]["baf"],
    asesor["dias"]["baf"]
)

evaluar_comisiones(asesor, porta_validas, mix_ok)

CONTROL COMERCIAL PARA ASESORES



Ingrese nombre del asesor:  tato



📅 Registro
Asesor: tato
Fecha: 26/01/2026 21:45

--- OBJETIVOS MENSUALES (configurables) ---


Objetivo Portabilidad:  28
Objetivo Línea Nueva:  2
Objetivo BAF:  5



--- PORTABILIDAD POR PLAN ---


...


--- VENTAS ACTIVADAS ---


Líneas nuevas activadas:  0
BAF activados:  0



--- DÍAS HÁBILES ---


Días hábiles (Portabilidad + Línea Nueva):  13
Días hábiles BAF:  13



📊 PORTABILIDAD
Total ventas: 0
4GB: 0
Otros planes: 0
Máx 4GB permitido (25%): 0
✅ Mix 4GB correcto
Ventas válidas: 0
Faltan: 28
Debe vender por día: 2.15
Semáforo: 🔴 ROJO

📊 LÍNEA NUEVA
Objetivo: 2
Activadas: 0
Faltan: 2
Debe vender por día: 0.15
Semáforo: 🟢 VERDE

📊 BAF
Objetivo: 5
Activados: 0
Faltan: 5
Debe vender por día: 0.38
Semáforo: 🟢 VERDE

EVALUACIÓN DE COMISIONES

🎯 COMISIÓN 60% + SÁBADOS LIBRES
❌ PORTABILIDAD: 0/17
❌ LINEA_NUEVA: 0/2
❌ BAF: 0/3
⛔ NO CUMPLE COMISIÓN

🎯 COMISIÓN 100% + SÁBADOS LIBRES
❌ PORTABILIDAD: 0/28
❌ LINEA_NUEVA: 0/2
❌ BAF: 0/5
⛔ NO CUMPLE COMISIÓN


In [6]:
!python -m nbconvert C_ca.ipynb --to html --no-input

[NbConvertApp] Converting notebook C_ca.ipynb to html
[NbConvertApp] Writing 272409 bytes to C_ca.html
